In [1]:
 from google.colab import drive
 
#  drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [2]:
import pandas as pd
from datetime import datetime, timedelta
from sklearn.linear_model import LogisticRegression
import joblib
import numpy as np


In [10]:
# Load the data into a pandas dataframe
df  = pd.read_csv("Boerschappen-Newdata-Box-Met-Events.csv")

C:\Users\LAPTOP WORLD\AppData\Local\Temp\ipykernel_10492\32576057.py:2: DtypeWarning: Columns (0,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df  = pd.read_csv("Boerschappen-Newdata-Box-Met-Events.csv")


In [13]:
df['event_timestamp'] = pd.to_datetime(df['event_timestamp'])
df['event_date'] = pd.to_datetime(df['event_timestamp'].dt.date)
df = df.sort_values(['customer_id', 'event_timestamp'])
df['last_event'] = df.groupby('customer_id')['event'].shift(1)
df['last_event_date'] = pd.to_datetime(df.groupby('customer_id')['event_date'].shift(1))
df['time_since_last_event'] = df['event_date'] - df['last_event_date']
df['time_since_last_event'] = df['time_since_last_event'].dt.days
df.head()

,customer_id,event,event_timestamp,nextBox,nextBoxDeliveryDate,nextBoxDeliveryTime,currentBoxPrice,currentBox,currentBoxDeliveryDate,currentBoxDeliveryTime,...,currentBoxDeliveryType,currentOrderStatus,nextBoxDeliveryType,nextBoxStatus,nextBoxStatusReason,nextOrderStatus,event_date,last_event,last_event_date,time_since_last_event
704,1,new order,2023-01-13 02:00:07,Fruittas,2023-01-16,16:00 - 20:00,12.50,Fruittas,2023-01-09,16:00 - 20:00,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-13,NaN,NaT,NaN
729,1,new order,2023-01-13 02:00:07,Fruittas,2023-01-16,16:00 - 20:00,12.50,Fruittas,2023-01-09,16:00 - 20:00,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-13,new order,2023-01-13,0.0
9810,1,new order,2023-01-13 02:00:07,Gemaksbox - 3p,2023-01-16,16:00 - 20:00,56.65,Gemaksbox - 3p,2023-01-09,16:00 - 20:00,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-13,new order,2023-01-13,0.0
9813,1,new order,2023-01-13 02:00:07,Gemaksbox - 3p,2023-01-16,16:00 - 20:00,56.65,Gemaksbox - 3p,2023-01-09,16:00 - 20:00,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-13,new order,2023-01-13,0.0
773,1,new order,2023-01-20 02:00:06,Fruittas,2023-01-23,16:00 - 20:00,12.50,Fruittas,2023-01-16,16:00 - 20:00,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-20,new order,2023-01-13,7.0


In [5]:
df['time_since_last_event']

48109    0.0
48599    0.0
48110    0.0
48111    0.0
48600    0.0
        ... 
48580    0.0
55825    6.0
48115    0.0
48086    7.0
48116    0.0
Name: time_since_last_event, Length: 113377, dtype: float64

In [6]:
# Update churned column based on time_since_last_event
df['churned'] = df['time_since_last_event'] >= 30
df.head(100)

<ipython-input-6-c6843e729505>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['churned'] = df['time_since_last_event'] >= 30


,customer_id,event,event_timestamp,event_date,last_event,last_event_date,time_since_last_event,churned
48109,1,new subscription,2022-09-26 14:40:00,2022-09-26,new subscription,2022-09-26,0.0,False
48599,1,new subscription,2022-09-26 14:40:00,2022-09-26,new subscription,2022-09-26,0.0,False
48110,1,new subscription,2022-09-26 14:41:00,2022-09-26,new subscription,2022-09-26,0.0,False
48111,1,new subscription,2022-09-26 14:41:00,2022-09-26,new subscription,2022-09-26,0.0,False
48600,1,new subscription,2022-09-26 14:41:00,2022-09-26,new subscription,2022-09-26,0.0,False
...,...,...,...,...,...,...,...,...
26428,6,new subscription,2023-01-19 15:18:00,2023-01-19,new order,2023-01-19,0.0,False
26435,6,new order,2023-01-20 02:00:00,2023-01-20,new subscription,2023-01-19,1.0,False
0,6,new order,2023-01-23 10:16:00,2023-01-23,new order,2023-01-20,3.0,False
1,6,new subscription,2023-01-23 10:16:00,2023-01-23,new order,2023-01-23,0.0,False


In [7]:
# Build logistic regression model
X = df[['time_since_last_event']]
y = df['churned']
model = LogisticRegression()
model.fit(X, y)


LogisticRegression()

In [8]:
# Save model
joblib.dump(model, 'churn_model.joblib')

['churn_model.joblib']

In [9]:
# Load model
loaded_model = joblib.load('churn_model.joblib')

In [15]:
# customer_id = 3481
customer_id = 3599
# Get the most recent event date for the customer
df_customer = df[df['customer_id'] == customer_id]
last_event_date = df_customer['event_timestamp'].max().date()

# Compute the time since the last event in days
time_since_last_event = (datetime.now().date() - last_event_date).days
new_data = pd.DataFrame({'time_since_last_event': [time_since_last_event]})
new_data.columns = ['time_since_last_event']


# Predict the probability of churn for the customer
churn_prob = loaded_model.predict_proba(new_data)[:,1][0]
churn = loaded_model.predict(new_data)
print("Probability:", churn_prob)
if time_since_last_event >= 30:
    # Compute the time until churn in days
    time_until_churn = int(np.ceil(30 / churn_prob))

    # Compute the churn date as the most recent event date plus the time until churn
    churn_date = last_event_date + timedelta(days=time_until_churn)

    # Check if churn date is in the past and adjust accordingly
    if churn_date < datetime.now().date():
        churn_date = datetime.now().date() + timedelta(days=1)

    print(f"Customer {customer_id} is likely to churn on {churn_date}")
else:
    print(f"Customer {customer_id} is not likely to churn in the near future")

Probability: 1.0
Customer 3599 is likely to churn on 2023-04-07
